<a href="https://colab.research.google.com/github/odarasimi/RealEstatePriceModel/blob/master/BangloreRealEstatePricePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import io
import pandas as pd
import matplotlib as pyplot
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)
import numpy as np
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
import random
import pickle

In [ ]:
from google.colab import files
bengaluru_prices = files.upload()

In [ ]:
df1 = pd.read_csv(io.BytesIO(bengaluru_prices["Bengaluru_House_Data.csv"]))
df1.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


**Drop a few irrelevant columns**

In [ ]:
df2 = df1.drop(["area_type", "society", "balcony", "availability"], 1)
df2.dropna(inplace=True)
df2["sizes"] = df2["size"].apply(lambda x: int(x.split(" ")[0]))
df2.drop(["size"], 1)

,location,total_sqft,bath,price,sizes
0,Electronic City Phase II,1056,2.0,39.07,2
1,Chikka Tirupathi,2600,5.0,120.00,4
2,Uttarahalli,1440,2.0,62.00,3
3,Lingadheeranahalli,1521,3.0,95.00,3
4,Kothanur,1200,2.0,51.00,2
...,...,...,...,...,...
13315,Whitefield,3453,4.0,231.00,5
13316,Richards Town,3600,5.0,400.00,4
13317,Raja Rajeshwari Nagar,1141,2.0,60.00,2
13318,Padmanabhanagar,4689,4.0,488.00,4


**Convert data in total_sq_ft to valid numerical values**

In [ ]:
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

In [ ]:
df2["total_sqft"] = df2["total_sqft"].apply(lambda x: convert_sqft_to_num(x))
df2.dropna(inplace=True)
df2["total_sqft"].unique()
df2['price_per_sqft'] = df2['price']*100000/df2['total_sqft']

In [ ]:
 
print(df2.shape)
df2.dropna(inplace=True)
print(df2.shape)
df2.head()

(13200, 7)
(13200, 7)


,location,size,total_sqft,bath,price,sizes,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250.000000


In [ ]:
len(df2["location"].unique())

1298

**Dimensionality reduction for the location column**

In [ ]:
df2["location"] = df2["location"].apply(lambda x: x.strip(" "))
location_stats = df2.groupby("location")["location"].agg("count").sort_values(ascending = False)
location_stats

location
Whitefield              533
Sarjapur  Road          392
Electronic City         304
Kanakpura Road          264
Thanisandra             235
                       ... 
Kumbhena Agrahara         1
Kudlu Village,            1
Konappana Agrahara        1
Kodanda Reddy Layout      1
1 Annasandrapalya         1
Name: location, Length: 1287, dtype: int64

In [ ]:
location_less_than_10 = location_stats[location_stats<=10]
df2.location = df2["location"].apply(lambda x: 'Other' if x in location_less_than_10 else x)
len(df2["location"].unique())

241

**Removing outliers**

In [ ]:
df_extract = pd.DataFrame()
for x, y in df2.groupby("location"):
  st = np.std(y.price_per_sqft)
  mean = np.mean(y.price_per_sqft)
  extract = y[(y.price_per_sqft>(mean-st)) & (y.price_per_sqft<=(mean+st))]
  df_extract = pd.concat([df_extract,extract],ignore_index=True)
df_extract.shape

(11434, 7)

In [ ]:
def remove_size_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby("location"):
        sizes_stats = {}
        for sizes, sizes_df in location_df.groupby("sizes"):
            sizes_stats[sizes] = {
                'mean': np.mean(sizes_df.price_per_sqft),
                'std': np.std(sizes_df.price_per_sqft),
                'count': sizes_df.shape[0]
            }
        for sizes, sizes_df in location_df.groupby("sizes"):
            stats = sizes_stats.get(sizes-1)
            if stats and stats["count"]>5:
                exclude_indices = np.append(exclude_indices, sizes_df[sizes_df.price_per_sqft<(stats["mean"])].index.values)
    return df.drop(exclude_indices,axis="index")
df_extract = remove_size_outliers(df_extract)
df_extract.shape


(7992, 7)

In [ ]:
df_extract2 = df_extract[df_extract.bath<df_extract.sizes+2]
df_extract2.drop(["size"], 1,inplace=True)
final_df = df_extract2

**Change values in location to valid numerical variables(onehot encoding)**

In [ ]:
dummies = pd.get_dummies(final_df.location)
dummies.drop("Other", 1, inplace=True)
final_df["location"]=np.array(dummies)      

In [ ]:
X_df = final_df.drop(["price"], 1)
final_df.head()

,location,total_sqft,bath,price,sizes,price_per_sqft
0,1,2850.0,4.0,428.0,4,15017.543860
1,1,1630.0,3.0,194.0,3,11901.840491
2,1,1200.0,6.0,125.0,6,10416.666667
3,1,1875.0,2.0,235.0,3,12533.333333
4,1,930.0,4.0,85.0,7,9139.784946


**Finally scaling the values and training the model**

In [ ]:
X = preprocessing.scale(np.array(final_df.drop(["price"],axis=1)))
y = np.array(final_df["price"])
(X.shape, y.shape)

((7891, 5), (7891,))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

0.8876770980933837

**K Fold cross Validation measuring accuracy**

In [ ]:
cv = ShuffleSplit(n_splits=5, test_size=0.2)
accuracy = cross_val_score(LinearRegression(), X, y, cv=cv)
print(np.mean(accuracy))

0.851783939855402


85.2% ACCURACY...NICE!...SORRT OF ;)